# I ) Computing the SVD

In [1]:
import numpy as np
import numpy.linalg as la


import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

## 1) For a square matrix

In [2]:
m = 4
n = m
A = np.random.randn(m, n)
print(A)

### Using numpy.linalg.svd

In [3]:
U, S, Vt = la.svd(A)

In [4]:
print(U)
print(U.shape)

In [5]:
print(Vt)
print(Vt.T.shape)

In [6]:
print(S)
print(S.shape)

### Using eigen-decomposition

Now compute the eigenvalues and eigenvectors of $A^TA$ as `eigvals` and `eigvecs`

In [7]:
eigvals, eigvecs = la.eig(A.T.dot(A))

Eigenvalues are real and positive. Coincidence?

In [8]:
eigvals

`eigvecs` are orthonormal! Check:

In [9]:
eigvecs.T @ eigvecs 

Now piece together the SVD:

In [10]:
S2 = np.sqrt(eigvals)
V2 = eigvecs
U2 = A @ V2 @ la.inv(np.diag(S2))

## 2) For a non-square square matrix

In [11]:
m = 3
n = 5
A = np.random.randn(m, n)
print(A)

You can obtain the SVD in the full format using `full_matrices=True` (full_matrices=True is the default value)

In [12]:
U, S, Vt = la.svd(A,full_matrices=True)

In [13]:
print(U)
print(U.shape)

print(Vt)
print(Vt.T.shape)

print(S)
print(S.shape)

Check the eigen decomposition:

In [14]:
eigvals, eigvecs = la.eig(A.T.dot(A))
print(eigvals)

In [15]:
eigvals.sort()

In [16]:
np.sqrt(eigvals[-3:])

Or you can use get the reduced form of the SVD:

In [17]:
U, S, Vt = la.svd(A,full_matrices=False)

In [18]:
print('A = ', A.shape)
print('U = ', U.shape)
print('S = ', S.shape)
print('V = ', Vt.T.shape)

## Relative cost of matrix factorizations

In [19]:
import numpy.linalg as npla
import scipy.linalg as spla
from time import time


In [20]:
n_values = np.logspace(1,3.5,10).astype(np.int32)
n_values

In [21]:
def matmat(A):
    A @ A

for name, f in [
        ("lu", spla.lu_factor),
        ("matmat", matmat),
        ("svd", npla.svd)
        ]:

    times = []
    print("----->", name)
    
    for n in n_values:
        A = np.random.randn(n, n)
        
        start_time = time()
        f(A)
        delta_time = time() - start_time
        times.append(delta_time)
        
        print("%d - %f" % (n, delta_time))
        
    plt.loglog(n_values, times, label=name)

plt.legend(loc="best")
plt.xlabel("Matrix size $n$")
plt.ylabel("Wall time [s]");
plt.grid();

# II ) SVD Applications

## 1) Rank of a matrix 

Creating matrices for the examples:

In [22]:
m = 6
n = 4
# Creating the orthogonal U and Vt matrices
X = np.random.randn(m, m)
U, _ = la.qr(X)
X = np.random.randn(n, n)
Vt, _ = la.qr(X)


# Creating the singular values
S = np.zeros((m,n))
# This creates a full rank matrix
r = min(m,n)
# This creates a rank deficient matrix
r = np.random.randint(1,min(m,n))

print("the rank of A is = ",r)
# Completing the singular value matrix Sigma
sig = np.random.randint(1,50,r)
sig.sort()
sigmas = sig[::-1]
for i,s in enumerate(sigmas):
    S[i,i] = s
    
print(S)

print(U)
    
print(Vt)

A = U@S@Vt

In [23]:
la.svd(A)

In [24]:
la.matrix_rank(A)

## 2) Low-rank approximations

#### Example 1:

In [27]:
with Image.open("quad.jpg") as img:
    rgb_img = np.array(img)
rgb_img.shape

In [28]:
img = np.sum(rgb_img, axis=-1)
img.shape

In [29]:
plt.figure(figsize=(20,10))
plt.imshow(img, cmap="gray")

In [30]:
u, sigma, vt = np.linalg.svd(img)
print('A = ', img.shape)
print('U = ', u.shape)
print('S = ', sigma.shape)
print('V.T = ', vt.shape)

In [31]:
plt.plot(sigma, lw=4)
plt.xlabel('singular value index')
plt.ylabel('singular values')

In [32]:
plt.loglog(sigma, lw=4)
plt.xlabel('singular value index')
plt.ylabel('singular values')

In [33]:
k = 0
img_k = sigma[k]*np.outer(u[:,k],vt[k,:])
plt.imshow(img_k, cmap="gray")

In [34]:
k = 1
img_k = sigma[k]*np.outer(u[:,k],vt[k,:])
plt.imshow(img_k, cmap="gray")

In [35]:
img_k = sigma[1]*np.outer(u[:,1],vt[1,:]) + sigma[2]*np.outer(u[:,2],vt[2,:])
plt.imshow(img_k, cmap="gray")

In [36]:
img_k = sigma[1]*np.outer(u[:,1],vt[1,:]) + sigma[4]*np.outer(u[:,4],vt[4,:])
plt.imshow(img_k, cmap="gray")

In [37]:
la.matrix_rank(img_k)

In [38]:
img_k = np.zeros(img.shape)
k=4
for i in range(k):
    img_k += sigma[i]*np.outer(u[:,i],vt[i,:])
plt.imshow(img_k, cmap="gray")

In [39]:
k=50
compressed_img = u[:,:k+1] @ np.diag(sigma[:k+1]) @ vt[:k+1,:]
plt.figure(figsize=(20,10))
plt.imshow(compressed_img, cmap="gray")

In [40]:
error = la.norm(img - compressed_img,2)
print(error)

In [41]:
sigma[k:k+3]

In [42]:
original_size = img.size
compressed_size = u[:,:k].size + sigma[:k].size + vt[:k,:].size
print("original size: %d" % original_size)
print("compressed size: %d" % compressed_size)
print("ratio: %f" % (compressed_size / original_size))

In [44]:
compressed_img = np.zeros(img.shape)
for k in range(500):
    compressed_img += sigma[k]*np.outer(u[:,k], vt[k,:])
    error = la.norm(img - compressed_img,2)
    if error < 4000:
        break
k    

#### Example 2:

In [45]:
with Image.open("Foellinger_Auditorium_2007.jpg") as img:
    rgb_img = np.array(img)
foellinger = np.sum(rgb_img, axis=-1)

def bestk(A, k):
    U,S,V = np.linalg.svd(A, full_matrices=False)
    return U[:, :k] @ np.diag(S[:k]) @ V[:k, :]

low_10 = bestk(foellinger, 10)
low_20 = bestk(foellinger, 20)
low_50 = bestk(foellinger, 50)

plt.figure(figsize=(10,10))
plt.subplot(2, 2, 1)
plt.imshow(low_10, cmap='gray')
plt.title("k = 10")
plt.subplot(2, 2, 2)
plt.imshow(low_20, cmap='gray')
plt.title("k = 20")
plt.subplot(2, 2, 3)
plt.imshow(low_50, cmap='gray')
plt.title("k = 50")
plt.subplot(2, 2, 4)
plt.imshow(foellinger, cmap='gray')
plt.title("k = 480")
plt.show()

## 3) Pseudo-inverse

### Square matrices:

In [46]:
m = 4
n = 4
# Creating the orthogonal U and Vt matrices
X = np.random.randn(m, m)
U, _ = la.qr(X)
X = np.random.randn(n, n)
Vt, _ = la.qr(X)


# Creating the singular values
S = np.zeros((m,n))
# This creates a full rank matrix
r = min(m,n)
# This creates a rank deficient matrix
r = np.random.randint(1,min(m,n))

print("the rank of A is = ",r)
# Completing the singular value matrix Sigma
sig = np.random.randint(1,50,r)
sig.sort()
sigmas = sig[::-1]
for i,s in enumerate(sigmas):
    S[i,i] = s

A = U@S@Vt

In [47]:
la.inv(A)

In [48]:
la.cond(A)

In [49]:
la.svd(A)

In [50]:
la.pinv(A)

In [51]:
u,s,vt = la.svd(A)

In [52]:
s

In [53]:
sinv = np.zeros(s.shape)
for i,si in enumerate(s):
    if si > 1e-10:
        sinv[i] = 1/si
sinv

In [54]:
vt.T@np.diag(sinv)@u.T

### Rectangular matrices

In [55]:
m = 6
n = 4
# Creating the orthogonal U and Vt matrices
X = np.random.randn(m, m)
U, _ = la.qr(X)
X = np.random.randn(n, n)
Vt, _ = la.qr(X)


# Creating the singular values
S = np.zeros((m,n))
# This creates a rank deficient matrix
r = np.random.randint(1,min(m,n))

print("the rank of A is = ",r)
# Completing the singular value matrix Sigma
sig = np.random.randint(1,50,r)
sig.sort()
sigmas = sig[::-1]
for i,s in enumerate(sigmas):
    S[i,i] = s

A = U@S@Vt

In [56]:
la.pinv(A)

In [57]:
u,s,vt = la.svd(A,full_matrices=False)
sinv = np.zeros(s.shape)
for i,si in enumerate(s):
    if si > 1e-10:
        sinv[i] = 1/si
vt.T@np.diag(sinv)@u.T

## 4) Matrix Norms and Condition number

### Square and non-singular matrices

In [58]:
m = 4
n = 4
# Creating the orthogonal U and Vt matrices
X = np.random.randn(m, m)
U, _ = la.qr(X)
X = np.random.randn(n, n)
Vt, _ = la.qr(X)


# Creating the singular values
S = np.zeros((m,n))
# This creates a full rank matrix
r = min(m,n)
# This creates a rank deficient matrix
## r = np.random.randint(1,min(m,n))

print("the rank of A is = ",r)
# Completing the singular value matrix Sigma
sig = np.random.randint(1,50,r)
sig.sort()
sigmas = sig[::-1]
for i,s in enumerate(sigmas):
    S[i,i] = s

A = U@S@Vt

Given the SVD of A...

In [59]:
u,s,vt = la.svd(A)
s

... determine the eucledian norm of $A$:

In [60]:
la.norm(A,2)

... determine the eucledian norm of $A^{-1}$

In [61]:
1/9

In [62]:
la.norm(la.inv(A),2)

... determine the condition number of $A$:

In [63]:
33/9

In [64]:
la.cond(A)

### Square and singular matrices

In [65]:
m = 4
n = 4
# Creating the orthogonal U and Vt matrices
X = np.random.randn(m, m)
U, _ = la.qr(X)
X = np.random.randn(n, n)
Vt, _ = la.qr(X)


# Creating the singular values
S = np.zeros((m,n))
# This creates a full rank matrix
r = min(m,n)
# This creates a rank deficient matrix
r = np.random.randint(1,min(m,n))

print("the rank of A is = ",r)
# Completing the singular value matrix Sigma
sig = np.random.randint(1,50,r)
sig.sort()
sigmas = sig[::-1]
for i,s in enumerate(sigmas):
    S[i,i] = s

A = U@S@Vt

Given the SVD of A...

In [66]:
u,s,vt = la.svd(A)
s

In [67]:
la.matrix_rank(A)

... determine the eucledian norm of $A$:

In [68]:
33

In [69]:
la.norm(A,2)

... determine the eucledian norm of $A^{+}$

In [70]:
1/13

In [71]:
la.norm(la.pinv(A),2)

... determine the condition number of $A$:

In [72]:
np.inf

In [73]:
la.cond(A)